In [ ]:
# Install networkx pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install networkx

import networkx as nx
import random
import matplotlib.pyplot as plt
import networkx.drawing as drawer
import copy
import itertools
from time import perf_counter
from datetime import datetime

In [ ]:
#Graph dict
totalGraphDict = {}

graphList = [(10,5), (15,8), (19,3)]

In [ ]:
#return a dict with key=(node1,node2) and value=weight12
def preprocessGraphToDict(graph):
    global totalGraphDict
    graphDict = {}
    for edge in graph.edges.data():
        left,right,weight = edge
        graphDict[(left,right)] = weight.get('weight')
        graphDict[(right,left)] = weight.get('weight')
    totalGraphDict = graphDict

In [ ]:
#apply double spanning tree algorithm to generate an initial solution. It returns a node list of the solution
def doubleSpanningTree(graph):
    #apply Kruskal
    T = nx.minimum_spanning_tree(graph,weight="weight")
    T_ordered = nx.dfs_tree(T, source=list(nx.nodes(graph))[0])
    return list(nx.nodes(T_ordered))

In [ ]:
#return a weighted graph with n nodes and each weight between minRange and maxRange
def generateCompleteGraphWeighted(n,minRange,maxRange):
    nodes = range(n)
    G = nx.empty_graph(nodes)
    if len(nodes) > 1:
        if G.is_directed():
            edges = itertools.permutations(nodes, 2)
        else:
            edges = itertools.combinations(nodes, 2)
            
        weightedEdges = []
        for edge in edges: 
            left,right = edge
            weightedEdge = (left,right,random.randint(minRange,maxRange))
            weightedEdges.append(weightedEdge)
            
        G.add_weighted_edges_from(weightedEdges)
    return G

In [ ]:
#Function that return a complete graph instance
def generateCompleteGraph(numberNodes,minRange=1,maxRange=1000):
    print('Generating complete graph')
    G = generateCompleteGraphWeighted(numberNodes,minRange,maxRange)
    print('Complete graph generated with ',numberNodes,' nodes and ',G.size(),' edges')
    return G

In [ ]:
#return a problem instance as a subset of the graphInstance graph. The problem is a random numberMachines nodes
def generateProblemInstance(graphInstance, numberMachines):
    #print('Generating random problem instance of ',numberMachines,' requests')
    problemInstance = graphInstance.subgraph(random.sample(range(0, len(graphInstance)), numberMachines))
    return problemInstance

In [ ]:
#return an initial solution of the problemInstance in O(n^2)
def generateInitialSolution(problemInstance,algo="hamilton"):
    if(algo=='hamilton'):
        initialSol = nx.algorithms.tournament.hamiltonian_path(problemInstance.to_directed())
    else:
        initialSol = doubleSpanningTree(problemInstance)
    #print(list(initialSol))
    return initialSol

In [ ]:
#return a graph that represents the shortest path between source and dest
def applySpt(source,dest,totalGraph,method='dijkstra'):
    #method accepted are bellman-ford or dijkstra
    pathList = nx.shortest_path(totalGraph, source=source, target=dest,weight='weight', method=method)
    return convertNodeListToWeightedGraph(pathList,False)

In [ ]:
#return a weighted graph from the currentSolutionList
def convertNodeListToWeightedGraph(currentSolutionList,addCycle=True):
    G = nx.empty_graph(currentSolutionList)
    weightedEdges = []
    for i in range(len(currentSolutionList)-1):
        currentNode = currentSolutionList[i]
        nextNode = currentSolutionList[i+1]
        #weightedEdge = findWeightedEdge(totalGraph,currentNode,nextNode)
        weightedEdgeTriple = (currentNode,nextNode,totalGraphDict[(currentNode,nextNode)])
        weightedEdges.append(weightedEdgeTriple)
    
    #cycleEdge = findWeightedEdge(totalGraph,currentSolutionList[-1],currentSolutionList[0])
    if(addCycle):
        cycleEdge = (currentSolutionList[-1],currentSolutionList[0],totalGraphDict[(currentSolutionList[-1],currentSolutionList[0])]) 
        weightedEdges.append(cycleEdge)

    G.add_weighted_edges_from(weightedEdges)
    return G

In [ ]:
#return a graph that is the totalGraph\currentSolution+{source and dest}
def prepareToSpt(source,dest,currentSolution,totalGraph):
    
    copiedCurrentSolution = currentSolution.copy()
        
    copiedTotalGraph = totalGraph.copy()
    copiedCurrentSolution.remove_node(source)
    copiedCurrentSolution.remove_node(dest)
                                      
    for i in nx.nodes(copiedCurrentSolution): 
        copiedTotalGraph.remove_node(i)
    return copiedTotalGraph

In [ ]:
#return the path cost of the current solution
def calculateSolutionCost(solution):
    total = 0
    for edge in solution.edges.data():
        left,right,weight = edge
        total += weight.get("weight")
    return total

In [ ]:
#return a new solution that is the addition of currentSolution and newPath minus the current arc between source and dest
#it is assumed that the newPath is better than the currentpath between source and dest
def addPathToCurrentSolution(source,dest,currentSolution,newPath):
    currentSolution.remove_edge(source, dest)
    for edge in newPath.edges.data():
        left,right,weight = edge
        if(left != source and left != dest):
            currentSolution.add_node(left)
        if(right != source and right != dest):
            currentSolution.add_node(right)
        currentSolution.add_edge(left, right, weight=weight.get("weight"))
        

In [ ]:
#apply LocalSearch algorithm; initialSolution and totalGraph are graphs;if maxIterations is -1 it will not be considered
def applyLocalSearch(initialSolution,totalGraph,maxIterations=-1,sptMethod='dijkstra',openedFile=None):
    remainingIteration = maxIterations
    foundImprovement = True
    currentSolution = convertNodeListToWeightedGraph(initialSolution)
    
    bestNewPath = None
    bestSubstitutedEdge = None
    bestImprovementScore = -1
    
    while remainingIteration != 0 and foundImprovement:
        remainingIteration -= 1
        foundImprovement = False
        for edge in currentSolution.edges.data():
            left,right,weight = edge
            preparedTotalGraph = prepareToSpt(left,right,currentSolution,totalGraph)
            newPath = applySpt(left,right,preparedTotalGraph,sptMethod)
            newPathScore = calculateSolutionCost(newPath)
            if(newPathScore < weight.get("weight")):
                #print("Found a new path for the edge ",left," - ",right)
                currentImprovement = weight.get("weight") - newPathScore
                if(currentImprovement > bestImprovementScore):
                    foundImprovement = True
                    bestNewPath = newPath
                    bestSubstitutedEdge = edge
                    bestImprovementScore = currentImprovement
        
        if(foundImprovement):
            left,right,weight = bestSubstitutedEdge
            addPathToCurrentSolution(left,right,currentSolution,bestNewPath)
            print("Added new path to current solution with an improvements of ",bestImprovementScore,"; newpath: ",list(nx.nodes(bestNewPath)))
            if(openedFile != None):
                openedFile.write("Added new path to current solution with an improvements of "+str(bestImprovementScore)+"; newpath: "+str(list(nx.nodes(bestNewPath)))+"\r\n")
        bestNewPath = None
        bestSubstitutedEdge = None
        bestImprovementScore = -1
    
    return currentSolution

In [ ]:
def drawGraph(graph):
    drawer.draw(graph, with_labels=True, font_weight='bold')

In [ ]:
graphInstances = []
problemInstances = []
for graphTest in graphList:
    nodes,requests = graphTest
    instanceGraph = generateCompleteGraph(nodes)
    graphInstances.append(instanceGraph)
    
    #generate problem instance
    problemInstance = generateProblemInstance(instanceGraph, requests)
    problemInstances.append(problemInstance)

currentProblemInstanceIndex = 0
for testInstance in graphInstances:

    #generate complete graph
    totalGraph = testInstance
    preprocessGraphToDict(totalGraph)
    print("\r\n\r\nComplete graph generated")

    #generate random problem instance
    problem = problemInstances[currentProblemInstanceIndex]
    currentProblemInstanceIndex += 1
    
    fileName = "N"+str(len(testInstance))+"R"+str(len(problem))+"-"+datetime.now().strftime("%d-%b-%Y_(%H-%M-%S)")+".txt"
    file = open(fileName,"w") 
    
    log ="############################\r\n\tTEST INSTANCE: \r\n\t\t#NODES: "+str(len(testInstance))+" \r\n\t\t#REQUESTS: "+str(len(problem))+"\r\n############################\r\n"
    print(log)
    file.write(log) 
    
    initialSolutionDst = generateInitialSolution(problem,algo='dst')
    log = "Initial solution DST generated with cost "+str(calculateSolutionCost(convertNodeListToWeightedGraph(initialSolutionDst)))+"\r\n"
    print(log)
    file.write(log)
    
    initialSolutionHam = generateInitialSolution(problem,algo='hamilton')
    log = "Initial solution Hamilton generated with cost "+str(calculateSolutionCost(convertNodeListToWeightedGraph(initialSolutionHam)))+"\r\n"
    print(log)
    file.write(log)
    
    log = "\r\n\r\n"
    print(log)
    file.write(log)
    
    
    #launching LocalSearch algorithm
    log = "Starting LocalSearch DIJ-DST...."
    print(log)
    file.write(log)
    
    t1_start = perf_counter()
    finalSolution = applyLocalSearch(initialSolutionDst,totalGraph,maxIterations=-1,sptMethod='dijkstra',openedFile=file)
    t1_stop = perf_counter() 
    log = "LocalSearch DIJ-DST end"
    print(log)
    file.write(log)
    log = "Final solution of cost (DIJ-DST): "+str(calculateSolutionCost(finalSolution))+" - ELAPSED: "+str(t1_stop-t1_start)+" seconds\r\n"
    print(log)
    file.write(log)
    log = "**********  "+str(finalSolution.edges.data())+"  **********\r\n\r\n"
    print(log)
    file.write(log)
    
    #launching LocalSearch algorithm
    log = "Starting LocalSearch DIJ-HAM...."
    print(log)
    file.write(log)
    
    t1_start = perf_counter()
    finalSolution = applyLocalSearch(initialSolutionHam,totalGraph,maxIterations=-1,sptMethod='dijkstra',openedFile=file)
    t1_stop = perf_counter()
    log = "LocalSearch DIJ-HAM end"
    print(log)
    file.write(log)
    
    log = "Final solution of cost (DIJ-HAM): "+str(calculateSolutionCost(finalSolution))+" - ELAPSED: "+str(t1_stop-t1_start)+" seconds\r\n"
    print(log)
    file.write(log)
    log = "**********  "+str(finalSolution.edges.data())+"  **********\r\n\r\n"
    print(log)
    file.write(log)
    
    #launching LocalSearch algorithm
    log = "Starting LocalSearch BELL-DST...."
    print(log)
    file.write(log)
    
    
    t1_start = perf_counter()
    finalSolution = applyLocalSearch(initialSolutionDst,totalGraph,maxIterations=-1,sptMethod='bellman-ford',openedFile=file)
    t1_stop = perf_counter()
    log = "LocalSearch BELL-DST end"
    print(log)
    file.write(log)
    log = "Final solution of cost (BELL-DST): "+str(calculateSolutionCost(finalSolution))+" - ELAPSED: "+str(t1_stop-t1_start)+" seconds\r\n"
    print(log)
    file.write(log)
    log = "**********  "+str(finalSolution.edges.data())+"  **********\r\n\r\n"
    print(log)
    file.write(log)
    
    #launching LocalSearch algorithm
    log = "Starting LocalSearch BELL-HAM...."
    print(log)
    file.write(log)
    
    t1_start = perf_counter()
    finalSolution = applyLocalSearch(initialSolutionHam,totalGraph,maxIterations=-1,sptMethod='bellman-ford',openedFile= file)
    t1_stop = perf_counter()
    log = "LocalSearch BELL-HAM end"
    print(log)
    file.write(log)
    log = "Final solution of cost (BELL-HAM): "+str(calculateSolutionCost(finalSolution))+" - ELAPSED: "+str(t1_stop-t1_start)+" seconds\r\n"
    print(log)
    file.write(log)
    log = "**********  "+str(finalSolution.edges.data())+"  **********\r\n\r\n"
    print(log)
    file.write(log)
    
    file.close()
    #drawGraph(finalSolution)